In [1]:
import pandas as pd
import numpy as np
import nltk
import spacy
import pickle

from flask import Flask, request, render_template
from gensim.models.ldamodel import LdaModel
from gensim import corpora
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nlp = spacy.load("en_core_web_sm")
nltk.download('vader_lexicon')

from absa_functions import *

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Evan\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
corpus = pickle.load(open('corpus.pkl', 'rb'))
dictionary = corpora.Dictionary.load('dictionary.gensim')
lda_model = LdaModel.load('best_lda_model.gensim')

In [3]:
sid = SentimentIntensityAnalyzer()

In [3]:
#numerical mapping for topics in LDA model
# topic_map = {0: 'menu', 1: 'service', 2: 'miscellaneous', 3: 'place', 4: 'price', 5: 'food', 6: 'staff'}



{0: 'menu',
 1: 'service',
 2: 'miscellaneous',
 3: 'place',
 4: 'price',
 5: 'food',
 6: 'staff'}

In [4]:
review = 'Wonderful music, beautiful restaurant, great wine.'

In [5]:
count = 0
doc = nlp(review)
for token in doc:
    count += 1
    if count <= 20:
    # if 12 <= count <= 18:
        print(token.text, token.dep_, token.head.text, token.head.pos_, token.pos_,[child for child in token.children])

Wonderful amod music NOUN ADJ []
music ROOT music NOUN NOUN [Wonderful, ,, restaurant, wine, .]
, punct music NOUN PUNCT []
beautiful amod restaurant NOUN ADJ []
restaurant appos music NOUN NOUN [beautiful, ,]
, punct restaurant NOUN PUNCT []
great amod wine NOUN ADJ []
wine appos music NOUN NOUN [great]
. punct music NOUN PUNCT []


In [6]:
parse_targets(nlp, review)

['music']

In [7]:
parse_adjectives(nlp, review)

['Wonderful', 'beautiful', 'great']

In [8]:
get_topic_from_word(prepare_text_for_lda('entitlement')[0], lda_model, TOPIC_MAP)

'miscellaneous'

In [9]:
def pos_prediction(restaurant_review):
    nlp = spacy.load("en_core_web_sm")
    targets = parse_targets(nlp, restaurant_review)
    adjectives = parse_adjectives(nlp, restaurant_review)

    outputs = []
    if len(targets) == len(adjectives): 
        for i in range(0, len(targets)):
            output = {}
            
            output.update({'aspect': targets[i], 'adjective': adjectives[i]})
            try:
                topic = get_topic_from_word(prepare_text_for_lda(targets[i])[0], lda_model, TOPIC_MAP)
            except:
                topic = 'miscellaneous'
            score = sid.polarity_scores(adjectives[i])['compound']
            sentiment = 'positive' if score > 0 else ('neutral' if score == 0 else 'negative')
            output.update({'topic': topic, 'polarity': sentiment})
            outputs.append(output)
    elif len(targets) > len(adjectives):
        for i in range(0, len(targets)):
            output = {}
            try:
                topic = get_topic_from_word(prepare_text_for_lda(targets[i])[0], lda_model, TOPIC_MAP)
                score = sid.polarity_scores(adjectives[i])['compound']
                sentiment = 'positive' if score > 0 else ('neutral' if score == 0 else 'negative')
                output.update({'aspect': targets[i], 'adjective': adjectives[i], 'topic': topic, 'polarity': sentiment})
            except IndexError:
                topic = get_topic_from_word(prepare_text_for_lda(targets[i])[0], lda_model, TOPIC_MAP)
                output.update({'aspect': targets[i], 'adjective': 'None', 'topic': topic, 'polarity': 'None'})
            
            outputs.append(output)
    elif len(targets) < len(adjectives):
        for i in range(0, len(adjectives)):
            output = {}
            try:
                topic = get_topic_from_word(prepare_text_for_lda(targets[i])[0], lda_model, TOPIC_MAP)
                score = sid.polarity_scores(adjectives[i])['compound']
                sentiment = 'positive' if score > 0 else ('neutral' if score == 0 else 'negative')
                output.update({'aspect': targets[i], 'adjective': adjectives[i], 'topic': topic, 'polarity': sentiment})
            except IndexError:
                score = sid.polarity_scores(adjectives[i])['compound']
                sentiment = 'positive' if score > 0 else ('neutral' if score == 0 else 'negative')
                output.update({'aspect': 'None', 'adjective': adjectives[i], 'topic': 'miscellaneous', 'polarity': sentiment})
            outputs.append(output)
    
    return pd.DataFrame(outputs)
        


In [10]:
pos_prediction(review)

,aspect,adjective,topic,polarity
0,music,Wonderful,menu,positive
1,None,beautiful,miscellaneous,positive
2,None,great,miscellaneous,positive


In [11]:
app = Flask(__name__)

@app.route('/')
def index():
    return "Hello, world!"
    